In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Słownik mapujący kategorie na ich URL
CATEGORY_URLS = {
    "Śniadania": "https://www.jadlonomia.com/przepisy/rodzaj_dania/sniadania/",
    "Przystawki": "https://www.jadlonomia.com/przepisy/rodzaj_dania/przystawki/",
    "Ciasta i desery": "https://www.jadlonomia.com/przepisy/rodzaj_dania/ciasta-i-desery/",
    "Do chleba": "https://www.jadlonomia.com/przepisy/rodzaj_dania/do-chleba/",
    "Zupy": "https://www.jadlonomia.com/przepisy/rodzaj_dania/zupy/",
    "Napoje": "https://www.jadlonomia.com/przepisy/rodzaj_dania/napoje/",
    "Lunche do pracy": "https://www.jadlonomia.com/przepisy/rodzaj_dania/lunche-do-pracy/",
    "Dania główne": "https://www.jadlonomia.com/przepisy/rodzaj_dania/dania-glowne/",
    "Sosy i dodatki": "https://www.jadlonomia.com/przepisy/rodzaj_dania/sosy-i-dodatki/",
}

def fetch_html(url):
    """Pobiera i zwraca obiekt BeautifulSoup dla podanego URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Błąd podczas pobierania strony: {url}.\nSzczegóły: {e}")
        return None

def get_recipe_links(category_url):
    """Zwraca listę linków do przepisów z danej kategorii."""
    recipes = []
    page = 1

    while True:
        paginated_url = f"{category_url}page/{page}/"
        soup = fetch_html(paginated_url)
        if not soup:
            break

        links_found = False
        for link in soup.find_all('a', href=True):
            href = link.get('href')
            if 'przepis' in href and href not in recipes:
                recipes.append(href)
                links_found = True

        if not links_found:
            break
        page += 1
    return recipes

def get_recipe_details(recipe_url, category):
    """Pobiera szczegóły przepisu oraz przypisuje kategorię."""
    try:
        response = requests.get(recipe_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Błąd podczas pobierania przepisu: {recipe_url}. Szczegóły: {e}")
        return None

    # Tytuł przepisu
    title = soup.find('h2', class_="h3").text.strip() if soup.find('h2', class_="h3") else "Nie znaleziono tytułu"

    # URL obrazka
    image_url = None
    image_section = soup.find('div', class_="text-center post-gallery")
    if image_section:
        image_tag = image_section.find('img')
        if image_tag and image_tag.get('src'):
            image_url = image_tag['src']

    # Składniki
    ingredients = []
    ingredients_section = soup.find("div", class_="font-bold subtitle")
    if ingredients_section:
        ingredients_block = ingredients_section.find_next_siblings("p")
        for p in ingredients_block:
            text = p.get_text(" ", strip=True)
            if text:
                ingredients.append(text)

    # Instrukcje
    instructions = []
    instructions_section = soup.find('div', class_="hyphenate")
    if instructions_section:
        instructions_block = instructions_section.find_next('ol')
        if instructions_block:
            li_tags = instructions_block.find_all("li")
            for li in li_tags:
                text = li.get_text(strip=True)
                if text:
                    instructions.append(text)

    return {
        "title": title,
        "category": category,  # Przypisujemy kategorię bezpośrednio
        "image_url": image_url,
        "ingredients": ingredients,
        "instructions": instructions,
    }

def save_to_csv(data, filename="recipes_with_categories.csv"):
    """Zapisuje dane w formacie CSV do pliku."""
    try:
        with open(filename, mode="w", encoding="utf-8", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["title", "category", "image_url", "ingredients", "instructions"])

            for recipe in data:
                writer.writerow([
                    recipe["title"],
                    recipe["category"],
                    recipe["image_url"],
                    "; ".join(recipe["ingredients"]),
                    "\n".join(recipe["instructions"]),
                ])
        print(f"Dane zapisane w pliku CSV: {filename}")
    except IOError as e:
        print(f"Błąd zapisu do pliku CSV: {e}")

# Główna logika
all_recipes = []
for category, category_url in CATEGORY_URLS.items():
    print(f"Przetwarzanie kategorii: {category}")
    recipe_links = get_recipe_links(category_url)
    for recipe_url in recipe_links:
        print(f"Pobieranie szczegółów przepisu: {recipe_url}")
        details = get_recipe_details(recipe_url, category)  # Przekazujemy kategorię
        if details:
            all_recipes.append(details)

# Zapisz dane do pliku CSV
save_to_csv(all_recipes)

Przetwarzanie kategorii: Śniadania
Błąd podczas pobierania strony: https://www.jadlonomia.com/przepisy/rodzaj_dania/sniadania/page/4/.
Szczegóły: 404 Client Error: Not Found for url: https://www.jadlonomia.com/przepisy/rodzaj_dania/sniadania/page/4/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/category/przepisy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/sniadania/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/do-chleba/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/lunche-do-pracy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/przystawki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/zupy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/dania-glowne/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/ciasta-i-desery/
Pobieran

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/jaglanka-z-kurkuma-i-pomaranczami/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/kokosowa-jaglanka-z-granate/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/granola-jabkowa/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/pieczone-jabka-z-jaglanka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/owsianka-z-masem-orzechowo-kokosowym/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/jagodowe-pancakes/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/pudding-chia-kokosowo-buraczany/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/dla-ochlody/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/dzien-dziecka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/burak/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/ko

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/truskawki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/kokosowy-p-udding-z-tapioki-z/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tofucznica-czyli-smazone-tofu-na/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/twarozek-roslinny-z-czosnkie/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/bezjajeczna-pasta-z-curry/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/wielkanoc/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowa-granola-czekoladowa/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/owsianka-orzechowo-cynamonowa/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/buraczane-pancakes/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/granola-pomaranczowo-kokosowa/
Pobieranie szczegółów przepisu: https://www.jadlonomi

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/pestki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/pomidor/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganskie-tzatziki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/ogorek/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/paprykowe-gazpacho-z-wedzona-papryka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/papryka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganska-feta-z-arbuzem/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/dla-ochlody/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/tofu/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/surowka-z-kapusty-i-migdalow/


Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/mloda-kaput/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/szparagowe-tabbouleh/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/szparagi/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/ekspresowe-szparagi-w-sosie-tahini/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/birmanska-salatka-z-imbiru/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/kameralny-sylwester/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/ciecierzyca/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tajskie-boczniaki-z-grilla/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/walentynki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/lososiowa-marchewka-na-trzy-sposoby/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepi

...

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/seleryba/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/lazanki-z-kiszona-kapusta-i-zurawina/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/dania-glowne/page/5/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/spaghetti-z-dyni/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/pesto-z-jarmuzu/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/powrot-do-szkoly/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/pieczone-pierogi-z-dynia-i-mase/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/curry-z-dynia-i-masem-nerkowcowy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/witarianskie-spaghetti-z-cukini/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/dla-ochlody/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przep

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rewolucyjne-prazone-pestki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/aquafaba/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/pestki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganski-majonez-doskonaly/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowe-smakowe-musztardy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/majowka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/kokosowy-bekon/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/niedzielne-gotowanie/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganski-sos-tatarski/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/cebula/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/grzyby/
Pobieranie szczegółów 

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/rodzaj_dania/sosy-i-dodatki/page/2/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganski-sos-pieczeniowy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/pietruszka/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/keczup-z-cukinii/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/cukinia/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/dyniowy-ajvar-z-olejem-arachidowy/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/dynia/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/co-do-chleba-salsa-z-pomidorow-i-oleju/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/awokado/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/grillowane-rzodkiewki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/s

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/guacamole-idealne/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/dla-ochlody/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/kameralny-sylwester/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowa-przyprawa-piernikowa/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/mikolajki/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowa-tahina/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/grusztarda/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowy-keczup-pikantny/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/domowy-keczup-agodny/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/idealny-majonez-bez-jajek/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/czekolada-z-daktyli/
Pobieranie szczegółów prze

Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/weganski-parmezan-z-migdaow/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/tag/tydzien-weganizmu/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/najprostszy-przepis-na-ajvar/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/pasta-z-pieczonej-papryki-z-mase/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/sos-z-fenkua-dla-tych-co-sie-boja-kopru/
Pobieranie szczegółów przepisu: https://www.jadlonomia.com/przepisy/skladnik/fenkul/
Dane zapisane w pliku CSV: recipes_with_categories.csv


In [ ]:
import pandas as pd

# Ścieżka do pliku (domyślnie w katalogu głównym po przesłaniu do Colab)
file_path = "/content/recipes_with_categories.csv"

# Wczytaj plik CSV do DataFrame
df = pd.read_csv(file_path)

# Wyświetl pierwsze 5 wierszy
df.tail(20)

,title,category,image_url,ingredients,instructions
1444,"Tom kha, czyli zupa kokosowa",Sosy i dodatki,NaN,NaN,NaN
1445,Za’atar,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,4 łyżki tymianku / całe opakowanie 2 łyżeczki ...,"Rozgrzać suchą patelnię, wysypać na nią sezam ..."
1446,Guacamole idealne,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,"2 dojrzałe dobrej jakości awokado, ja kupuję s...","Przeciąć awokado na pół, obrócić połowkę, pozb..."
1447,Wegańska feta z arbuzem,Sosy i dodatki,NaN,NaN,NaN
1448,Tofu zmieniające życie,Sosy i dodatki,NaN,NaN,NaN
1449,Domowa przyprawa piernikowa,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,"5 niedużych lasek cynamonu, 30 g 5 łyżeczek su...",Wild Tofupisze:3 grudnia 2013 o 01:36Anyż zale...
1450,Ekspresowe prezenty last minute,Sosy i dodatki,NaN,NaN,NaN
1451,Domowa tahina,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,250 g łuskanego sezamu ewentualnie 2 łyżki ole...,"Rozgrzać suchą patelnię, wsypać sezam i prażyć..."
1452,Grusztarda,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,1 kg dojrzałych gruszek ⅓ szklanki octu jabłko...,Piekarnik rozgrzać do 180 stopni. Gruszki obra...
1453,Domowy keczup pikantny,Sosy i dodatki,https://www.jadlonomia.com/wp-content/uploads/...,"3 kg pomidorów, z wykrojonymi gniazdami nasien...",Na dnie dużego garnka (przynajmniej 5 litroweg...


In [ ]:
df.describe()

,title,category,image_url,ingredients,instructions
count,1464,1464,846,792,848
unique,601,9,593,558,595
top,"Tom kha, czyli zupa kokosowa",Dania główne,https://www.jadlonomia.com/wp-content/uploads/...,2 ½ szklanki startego kalafiora / ½ kalafiora ...,Ryż zalać sporą ilością zimnej wody i zostawić...
freq,56,257,4,4,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1464 non-null   object
 1   category      1464 non-null   object
 2   image_url     846 non-null    object
 3   ingredients   792 non-null    object
 4   instructions  848 non-null    object
dtypes: object(5)
memory usage: 57.3+ KB


In [ ]:
import os
# Wyświetl wszystkie pliki w katalogu głównym Colab
print(os.listdir('/content'))

['.config', 'recipes_jadlonomia.csv', 'recipes_jadlonomia (1) (2).json', 'recipes_jadlonomia.json', 'recipes_jadlonomia (1).json', 'recipes_jadlonomia (1) (1).json', 'recipes_with_categories.csv', 'sample_data']


In [ ]:
from google.colab import files

# Pobierz plik na lokalny komputer
files.download("/content/recipes_with_categories.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Usuwanie wierszy z brakującymi danymi
filtered_df = df[
    (df["image_url"].notna()) &  # Obrazek nie jest None
    (df["ingredients"].apply(len) > 0) &  # Składniki nie są puste
    (df["instructions"].apply(len) > 0)  # Instrukcje nie są puste
]

# Wyświetlenie przefiltrowanych danych
filtered_df.head()

,title,image_url,ingredients,instructions
15,Jogurtowy sernik,https://www.jadlonomia.com/wp-content/uploads/...,"[Na spód: 180 g ciastek, owsianych, herbatnikó...",[Dzień wcześniej:durszlak wyłożyć podwójnie zł...
17,"Tom kha, czyli zupa kokosowa",https://www.jadlonomia.com/wp-content/uploads/...,"[4 cm imbiru 3 łodygi trawy cytrynowej, 2 pusz...","[Jeśli zupa ma być podana jako cały obiad, zac..."
23,Racuchy pączkowe,https://www.jadlonomia.com/wp-content/uploads/...,"[1 1/2 szklanki mleka roślinnego, 2 1/4 szklan...",[W piekarniku zapalić lampkę – dzięki temu lek...
25,Postępowy sos pomidorowy,https://www.jadlonomia.com/wp-content/uploads/...,"[3 – 4 ząbki czosnku 3 łyżki oliwy, 150 g pomi...",[Przygotować składniki sosu. Czosnek obrać i z...
27,Tofuryba świąteczna,https://www.jadlonomia.com/wp-content/uploads/...,"[Składniki na wywar:, 1/2 litra bulionu 2 – 3 ...",[W garnku połączyć wszystkie składniki wywaru....


In [ ]:
filtered_df.describe()

,title,image_url,ingredients,instructions
count,565,565,565,565
unique,564,565,565,565
top,Majonez bez jajek,https://www.jadlonomia.com/wp-content/uploads/...,"[Na spód: 180 g ciastek, owsianych, herbatnikó...",[Dzień wcześniej:durszlak wyłożyć podwójnie zł...
freq,2,1,1,1
